# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Feb 15 2023 10:20AM,257437,15,PNC428856A,"Person & Covey, Inc.",Released
1,Feb 15 2023 10:20AM,257437,15,PNC415436B,"Person & Covey, Inc.",Released
2,Feb 15 2023 10:20AM,257437,15,PNC415437C,"Person & Covey, Inc.",Released
3,Feb 15 2023 10:20AM,257437,15,PNC415438B,"Person & Covey, Inc.",Released
4,Feb 15 2023 10:11AM,257436,12,7597425,"Signd, Inc.",Released
5,Feb 15 2023 10:09AM,257435,10,PRF-47460,Bio-PRF,Released
6,Feb 15 2023 9:55AM,257433,21,641486,"NBTY Global, Inc.",Released
7,Feb 15 2023 9:55AM,257432,21,641485,"NBTY Global, Inc.",Released
8,Feb 15 2023 9:43AM,257429,10,CLI00469.1,"CLINUVEL, Inc.",Released
9,Feb 15 2023 9:43AM,257429,10,CLI00470.1,"CLINUVEL, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
46,257432,Released,1
47,257433,Released,1
48,257435,Released,1
49,257436,Released,1
50,257437,Released,4


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
257432,NaN,NaN,1.0
257433,NaN,NaN,1.0
257435,NaN,NaN,1.0
257436,NaN,NaN,1.0
257437,NaN,NaN,4.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
257339,0.0,1.0,0.0
257340,31.0,10.0,3.0
257341,0.0,0.0,2.0
257342,0.0,1.0,4.0
257346,0.0,0.0,7.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
257339,0,1,0
257340,31,10,3
257341,0,0,2
257342,0,1,4
257346,0,0,7


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,257339,0,1,0
1,257340,31,10,3
2,257341,0,0,2
3,257342,0,1,4
4,257346,0,0,7


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,257339,,1,
1,257340,31,10,3
2,257341,,,2
3,257342,,1,4
4,257346,,,7


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Feb 15 2023 10:20AM,257437,15,"Person & Covey, Inc."
4,Feb 15 2023 10:11AM,257436,12,"Signd, Inc."
5,Feb 15 2023 10:09AM,257435,10,Bio-PRF
6,Feb 15 2023 9:55AM,257433,21,"NBTY Global, Inc."
7,Feb 15 2023 9:55AM,257432,21,"NBTY Global, Inc."
8,Feb 15 2023 9:43AM,257429,10,"CLINUVEL, Inc."
12,Feb 15 2023 9:37AM,257428,10,"Citieffe, Inc."
13,Feb 15 2023 9:28AM,257425,15,"Alliance Pharma, Inc."
43,Feb 15 2023 9:27AM,257423,16,"Emersa Waterbox, LLC"
44,Feb 15 2023 9:26AM,257398,15,"Alliance Pharma, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Feb 15 2023 10:20AM,257437,15,"Person & Covey, Inc.",,,4
1,Feb 15 2023 10:11AM,257436,12,"Signd, Inc.",,,1
2,Feb 15 2023 10:09AM,257435,10,Bio-PRF,,,1
3,Feb 15 2023 9:55AM,257433,21,"NBTY Global, Inc.",,,1
4,Feb 15 2023 9:55AM,257432,21,"NBTY Global, Inc.",,,1
5,Feb 15 2023 9:43AM,257429,10,"CLINUVEL, Inc.",,,4
6,Feb 15 2023 9:37AM,257428,10,"Citieffe, Inc.",,,1
7,Feb 15 2023 9:28AM,257425,15,"Alliance Pharma, Inc.",,,30
8,Feb 15 2023 9:27AM,257423,16,"Emersa Waterbox, LLC",,,1
9,Feb 15 2023 9:26AM,257398,15,"Alliance Pharma, Inc.",,,16


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 15 2023 10:20AM,257437,15,"Person & Covey, Inc.",4,,
1,Feb 15 2023 10:11AM,257436,12,"Signd, Inc.",1,,
2,Feb 15 2023 10:09AM,257435,10,Bio-PRF,1,,
3,Feb 15 2023 9:55AM,257433,21,"NBTY Global, Inc.",1,,
4,Feb 15 2023 9:55AM,257432,21,"NBTY Global, Inc.",1,,
5,Feb 15 2023 9:43AM,257429,10,"CLINUVEL, Inc.",4,,
6,Feb 15 2023 9:37AM,257428,10,"Citieffe, Inc.",1,,
7,Feb 15 2023 9:28AM,257425,15,"Alliance Pharma, Inc.",30,,
8,Feb 15 2023 9:27AM,257423,16,"Emersa Waterbox, LLC",1,,
9,Feb 15 2023 9:26AM,257398,15,"Alliance Pharma, Inc.",16,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 15 2023 10:20AM,257437,15,"Person & Covey, Inc.",4,,
1,Feb 15 2023 10:11AM,257436,12,"Signd, Inc.",1,,
2,Feb 15 2023 10:09AM,257435,10,Bio-PRF,1,,
3,Feb 15 2023 9:55AM,257433,21,"NBTY Global, Inc.",1,,
4,Feb 15 2023 9:55AM,257432,21,"NBTY Global, Inc.",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 15 2023 10:20AM,257437,15,"Person & Covey, Inc.",4.0,NaN,NaN
1,Feb 15 2023 10:11AM,257436,12,"Signd, Inc.",1.0,NaN,NaN
2,Feb 15 2023 10:09AM,257435,10,Bio-PRF,1.0,NaN,NaN
3,Feb 15 2023 9:55AM,257433,21,"NBTY Global, Inc.",1.0,NaN,NaN
4,Feb 15 2023 9:55AM,257432,21,"NBTY Global, Inc.",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 15 2023 10:20AM,257437,15,"Person & Covey, Inc.",4.0,0.0,0.0
1,Feb 15 2023 10:11AM,257436,12,"Signd, Inc.",1.0,0.0,0.0
2,Feb 15 2023 10:09AM,257435,10,Bio-PRF,1.0,0.0,0.0
3,Feb 15 2023 9:55AM,257433,21,"NBTY Global, Inc.",1.0,0.0,0.0
4,Feb 15 2023 9:55AM,257432,21,"NBTY Global, Inc.",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,7206712,180.0,32.0,31.0
12,514854,1.0,1.0,0.0
15,1029614,56.0,0.0,1.0
16,257423,1.0,0.0,0.0
19,1286925,4.0,2.0,0.0
21,514865,2.0,0.0,0.0
22,772178,2.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,7206712,180.0,32.0,31.0
1,12,514854,1.0,1.0,0.0
2,15,1029614,56.0,0.0,1.0
3,16,257423,1.0,0.0,0.0
4,19,1286925,4.0,2.0,0.0
5,21,514865,2.0,0.0,0.0
6,22,772178,2.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,180.0,32.0,31.0
1,12,1.0,1.0,0.0
2,15,56.0,0.0,1.0
3,16,1.0,0.0,0.0
4,19,4.0,2.0,0.0
5,21,2.0,0.0,0.0
6,22,2.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,180.0
1,12,Released,1.0
2,15,Released,56.0
3,16,Released,1.0
4,19,Released,4.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,21,22
Status,,,,,,,
Completed,31.0,0.0,1.0,0.0,0.0,0.0,0.0
Executing,32.0,1.0,0.0,0.0,2.0,0.0,1.0
Released,180.0,1.0,56.0,1.0,4.0,2.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,21,22
0,Completed,31.0,0.0,1.0,0.0,0.0,0.0,0.0
1,Executing,32.0,1.0,0.0,0.0,2.0,0.0,1.0
2,Released,180.0,1.0,56.0,1.0,4.0,2.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,21,22
0,Completed,31.0,0.0,1.0,0.0,0.0,0.0,0.0
1,Executing,32.0,1.0,0.0,0.0,2.0,0.0,1.0
2,Released,180.0,1.0,56.0,1.0,4.0,2.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()